In [ ]:
import time

import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib import style

from more_itertools import chunked

from google.cloud import storage

In [1]:
# Configuration constants
VALIDATION_RATIO = 0.1

In [ ]:
client = storage.Client()

bucket_name = "tdt4173-datasets"
bucket = client.get_bucket(bucket_name)

blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

In [ ]:
blob_name = "cats-vs-dogs/processed/catsvsdogs.npy"
blob = bucket.get_blob(blob_name)
data_file = "data/processed/catsdogs-2.npy"
blob.download_to_filename(data_file)

In [ ]:
training_data = np.load(data_file, allow_pickle=True)

In [ ]:
len(training_data)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

In [ ]:
x = torch.Tensor([i[0] for i in training_data]).view(-1, 128, 128)
x /= 255.0
y = torch.Tensor([i[1] for i in training_data])

In [ ]:
class AnkileNetV1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.conv3 = nn.Conv2d(64, 128, 5, padding=2)
        
        # Input neurons defined by
        # out_channels * (input_img_size / (pool_size**num_pooling)**2)
        self.fc1 = nn.Linear(int(128*(128 / 2**3)**2), 512)
        self.fc2 = nn.Linear(512, 2)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        
        x = x.view(-1, int(128*(128 / 2**3)**2))
        
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        
        return x;

In [ ]:
val_size = int(len(x) * VALIDATION_RATIO)
print(val_size)

In [ ]:
train_x = x[:-val_size]
train_y = y[:-val_size]

In [ ]:
def fwd_pass(x, y, train=False):

    if train:
        net.zero_grad()
    out = net(x)
    acc = np.mean([int(torch.argmax(y_pred) == torch.argmax(y_real)) for y_pred, y_real in zip(out, y)])
    loss = loss_function(out, y)

    if train:
        loss.backward()
        optimizer.step()

    return acc, loss

In [ ]:
test_x = x[-val_size:].to(device)
test_y = y[-val_size:].to(device)

In [ ]:
def test(size=32):
    X, y = test_x[:size], test_y[:size]
    val_acc, val_loss = fwd_pass(X.view(-1, 1, 128, 128).to(device), y.to(device))
    return val_acc, val_loss

In [ ]:
MODEL_NAME = f"AnkileNetV1-{int(time.time())}"
print(f"Model name: {MODEL_NAME}")

net = AnkileNetV1().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

def train(net):
    BATCH_SIZE = 1
    EPOCHS = 5

    with open("model.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
                batch_x = train_x[i:i+BATCH_SIZE].view(-1, 1, 128, 128).to(device)
                batch_y = train_y[i:i+BATCH_SIZE].to(device)

                acc, loss = fwd_pass(batch_x, batch_y, train=True)

                if i % 100 == 0:
                    val_acc, val_loss = test(size=100)
                    f.write(f"{MODEL_NAME},{round(time.time(),3)},{round(float(acc),2)},{round(float(loss), 4)},{round(float(val_acc),2)},{round(float(val_loss),4)}\n")

                    
            print(f"Epoch: {epoch}. Loss: {loss}.")
train(net)

In [ ]:
style.use("ggplot")

model_name = "AnkileNetV1-1604242949"


def create_acc_loss_graph(model_name):
    times = []
    accs = []
    losses = []
    
    val_accs = []
    val_losses = []
    
    with open("model.log", "r") as f:
        
        for line in f.readlines():
            name, time, acc, loss, val_acc, val_loss = line.split(",")
            
            times.append(float(time))
            accs.append(float(acc))
            losses.append(float(loss))
            
            val_accs.append(float(val_acc))
            val_losses.append(float(val_loss))
            
    fig = plt.figure()
    
    ax1 = plt.subplot2grid((2, 1), (0, 0))
    ax2 = plt.subplot2grid((2, 1), (1, 0), sharex=ax1)
    
    ax1.plot(times, accs, label="acc")
    ax1.plot(times, val_accs, label="val_acc")
    ax1.legend(loc=2)
    
    ax2.plot(times, losses, label="loss")
    ax2.plot(times, val_losses, label="val_loss")
    ax2.legend(loc=2)
    plt.show()

create_acc_loss_graph(model_name)

In [ ]:
# Not currently in use, I think
correct = 0

with torch.no_grad():
    for i, y_real in enumerate(tqdm(test_y)):
        real_class = torch.argmax(y_real)
        pred_class = torch.argmax(net(test_x[i].view(-1, 1, 128, 128))[0])
        
        correct += int(real_class == pred_class)
        
print(f"Accuracy: {round(correct / len(test_x), 3)}")

In [ ]:
for im in test_x:
    pred = int(torch.argmax(net(im.view(-1, 1, 128, 128))))
    convert = {0: "Cat", 1: "Dog"}
    print(f"Net predicted it is `{convert[pred]}`")
    
    plt.imshow(im.cpu(), cmap="gray")
    plt.pause(0.05)
    print("="*50)